In [ ]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [1]:
import pandas as pd
from keras.callbacks import History, ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
import os
import numpy as np
from data_analysis import calculate_metrics
from functools import partial
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import pickle
import dill
from hyper_mining import objective_fn
import xgboost as xgb

Using TensorFlow backend.
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\tomas\miniconda3\envs\binding

In [2]:
fspace = {
    'conv1' : hp.quniform('conv1', 32, 64, 8),
    'conv2' : hp.quniform('conv2', 64, 128, 8),
    'conv3' : hp.quniform('conv3', 128, 168, 8),
    'fp' : hp.quniform('fp', 96, 196, 8),
    'dense1' : hp.quniform('dense1',96,512,32),
    'dense2' : hp.quniform('dense2',96,512,32),
    'dense3' : hp.quniform('dense3',64,512,32),
    'dropout_rate' : hp.uniform('dropout_rate',0.1,0.5),
    'lr' : hp.uniform('lr',0.000001,0.01),
    'n_epochs' : hp.quniform('n_epochs',15,60,5),
    'batch_size' : hp.quniform('batch_size',64,256,16),
    'colsample_bylevel' : hp.uniform('colsample_bylevel', 0.1, 1), 
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.1, 1), 
    'gamma' : hp.uniform('gamma', 0.1, 1), 
    'learning_rate' : hp.uniform('learning_rate', 0.1, 1),
    'max_delta_step' : hp.quniform('max_delta_step',1,10,1),
    'max_depth' : hp.quniform('max_depth',6, 12, 1),
    'min_child_weight' : hp.quniform('min_child_weight',10 ,500 ,5),
    'reg_alpha' : hp.uniform('reg_alpha',0.1,100),
    'reg_lambda' : hp.uniform('reg_lambda',0.1,100),
    'subsample' : hp.uniform('subsample',0.1,1.0),
    'max_bin' : hp.quniform('max_bin',16,256,16)
    #'margin' : hp.uniform('margin',0.2,2)
}

In [3]:
target = 'p38'
base_path = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath = base_path+f'/data/{target}/data.csv'
df=pd.read_csv(data_fpath).set_index('biolab_index')

with open(base_path+f'/data/{target}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds = dill.load(in_f)
with open(base_path+f'/data/{target}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds = dill.load(in_f)

In [4]:
training_list = [df.loc[train_val_folds[0][0]],
                 df.loc[train_val_folds[1][0]],
                 df.loc[train_val_folds[2][0]],
                 df.loc[train_val_folds[3][0]],
                 df.loc[train_val_folds[4][0]],
                 df.loc[train_val_folds[5][0]],
                 df.loc[train_test_folds[0]]
                 ]
validation_list = [df.loc[train_val_folds[0][1]],
                   df.loc[train_val_folds[1][1]],
                   df.loc[train_val_folds[2][1]],
                   df.loc[train_val_folds[3][1]],
                   df.loc[train_val_folds[4][1]],
                   df.loc[train_val_folds[5][1]],
                   df.loc[train_test_folds[1]]
                   ]

In [5]:
fmin_objective = partial(objective_fn, train_sets = training_list, val_sets = validation_list)

In [6]:
def run_trials():

    trials_step = 0  # how many additional trials to do after loading saved trials. 1 = save after iteration
    max_trials = 1  # initial max_trials. put something small to not have to wait

    
    try:  # try to load an already saved trials object, and increase the max
        trials = pickle.load(open("gcn_xgb.hyperopt", "rb"))
        print("Found saved Trials! Loading...")
        max_trials = len(trials.trials) + trials_step
        print("Rerunning from {} trials to {} (+{}) trials".format(len(trials.trials), max_trials, trials_step))
    except:  # create a new trials object and start searching
        trials = Trials()

    best = fmin(fn = fmin_objective, space = fspace, algo=tpe.suggest, max_evals=max_trials, trials=trials)

    print("Best:", best)
    
    # save the trials object
    with open("gcn_xgb.hyperopt", "wb") as f:
        pickle.dump(trials, f)
    return(trials)

In [7]:
trials = run_trials()

Found saved Trials! Loading...
Rerunning from 154 trials to 154 (+0) trials
100%|████████████████████████████████████████████████████████████████████████| 154/154 [00:00<?, ?trial/s, best loss=?]
Best: {'batch_size': 288.0, 'colsample_bylevel': 0.4371082812232264, 'colsample_bytree': 0.4179415558635843, 'conv1': 56.0, 'conv2': 88.0, 'conv3': 136.0, 'dense1': 384.0, 'dense2': 288.0, 'dense3': 224.0, 'dropout_rate': 0.27225175676555935, 'fp': 152.0, 'gamma': 0.919836526180396, 'learning_rate': 0.41409388868400826, 'lr': 0.0008110012706176706, 'max_bin': 48.0, 'max_delta_step': 2.0, 'max_depth': 7.0, 'min_child_weight': 20.0, 'n_epochs': 25.0, 'reg_alpha': 42.8887552483495, 'reg_lambda': 12.306130216692438, 'subsample': 0.6038298323514097}


In [8]:
best_loss = trials.trials[0]['result']['loss']
for i in range(1,len(trials.trials)):
    if (trials.trials[i]['result']['loss'] <=  best_loss):
        best_loss = trials.trials[i]['result']['loss']
        index = i
best_params = trials.trials[index]['misc']['vals']

In [10]:
from hyper_mining import XGB_predictor,GCN_online_mining_test
from data_analysis import calculate_metrics
es = EarlyStopping(monitor='loss',patience=8, min_delta=0)
rlr = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=4, verbose=1, min_lr=0.0000001)
gcn_best = {
        "num_layers" : 3,
        "max_atoms" : 70,
        "num_atom_features" : 62,
        "num_atom_features_original" : 62,
        "num_bond_features" : 6,
        "max_degree" : 5,
        "conv_width" : [int(best_params['conv1'][0]), int(best_params['conv2'][0]), int(best_params['conv3'][0])],
        "fp_length" : [int(best_params['fp'][0]), int(best_params['fp'][0]), int(best_params['fp'][0])],
        "activ_enc" : "selu",
        "activ_dec" : "selu",
        "learning_rates" : [0.001,0.001,0.001],
        "learning_rates_fp": [0.005,0.005,0.005],
        "losses_conv" : {
                    "neighbor_output": "mean_squared_error",
                    "self_output": "mean_squared_error",
                    },
        "lossWeights" : {"neighbor_output": 1.0, "self_output": 1.0},
        "metrics" : "mse",
        "loss_fp" : "mean_squared_error",
        "enc_layer_names" : ["enc_1", "enc_2", "enc_3"],
        'callbacks' : [es,rlr],
        'adam_decay': 0.0005329142291371636,
        'beta': 5,
        'p': 0.004465204118126482,
        'dense_size' : [int(best_params['dense1'][0]), int(best_params['dense2'][0]), int(best_params['dense3'][0])],
        'dropout_rate' : [best_params['dropout_rate'][0], best_params['dropout_rate'][0]],
        'lr' : best_params['lr'][0],
        'batch_size' : int(best_params['batch_size'][0]),
        'n_epochs' : int(best_params['n_epochs'][0])
        #'margin' : best_params['margin'][0]
        }
xgb_best = {
        "colsample_bylevel" : best_params['colsample_bylevel'][0],
        "colsample_bytree" : best_params['colsample_bytree'][0],
        "gamma" : best_params['gamma'][0],
        "eta" : best_params['learning_rate'][0],
        "max_delta_step" : int(best_params['max_delta_step'][0]),
        "max_depth" : int(best_params['max_depth'][0]),
        "min_child_weight" : int(best_params['min_child_weight'][0]),
        "alpha" : best_params['reg_alpha'][0],
        "lambda" : best_params['reg_lambda'][0],
        "subsample" : best_params['subsample'][0],
        "max_bin" : int(best_params['max_bin'][0]),
        "eval_metric":'auc',
        "objective":'binary:logistic',
        "booster":'gbtree'
        #"single_precision_histogram" : True
        }
class_XGB = XGB_predictor(xgb_best)
class_GCN = GCN_online_mining_test(gcn_best)

In [11]:
xgb_best

{'colsample_bylevel': 0.4371082812232264,
 'colsample_bytree': 0.4179415558635843,
 'gamma': 0.919836526180396,
 'eta': 0.41409388868400826,
 'max_delta_step': 2,
 'max_depth': 7,
 'min_child_weight': 20,
 'alpha': 42.8887552483495,
 'lambda': 12.306130216692438,
 'subsample': 0.6038298323514097,
 'max_bin': 48,
 'eval_metric': 'auc',
 'objective': 'binary:logistic',
 'booster': 'gbtree'}

In [10]:
#K.clear_session()
training_metrics = {}
validation_metrics = {}
es2 = EarlyStopping(monitor='loss',patience=15, min_delta=0)
rlr2 = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=2, verbose=1, min_lr=0.000000001)
for i in range(len(training_list)):
        X_atoms_cold,X_bonds_cold,X_edges_cold = class_GCN.dataframe_to_gcn_input(validation_list[i])
        Y_cold = validation_list[i].Binary 
        Y_dummy_cold = np.empty((X_atoms_cold.shape[0],gcn_best['dense_size'][2]+1))
        X_atoms_train, X_bonds_train, X_edges_train = class_GCN.dataframe_to_gcn_input(training_list[i])
        Y = training_list[i].Binary
        Y_dummy_train = np.empty((X_atoms_train.shape[0],gcn_best['dense_size'][2]+1))
        
        gcn_encoder = class_GCN.build_encoder()
        gcn_model = class_GCN.build_model(gcn_encoder)
        gcn_mining = class_GCN.build_mining(gcn_model)
        
        gcn_mining.fit([X_atoms_train,X_bonds_train,X_edges_train,Y],
                       Y_dummy_train,
                       epochs = gcn_best['n_epochs'],
                       batch_size = gcn_best['batch_size'],
                       shuffle = True,
                       validation_data = ([X_atoms_cold,X_bonds_cold,X_edges_cold,Y_cold],Y_dummy_cold),
                       callbacks=[es2,rlr2]
                      )
        #Predict Embeddings
        embeddings_cold = gcn_model.predict([X_atoms_cold,X_bonds_cold,X_edges_cold])
        embeddings_train = gcn_model.predict([X_atoms_train, X_bonds_train, X_edges_train])
        
        #Prepare data for XGBoost
        dmatrix_train = class_XGB.to_xgb_input(Y,embeddings_train)
        dmatrix_cold = class_XGB.to_xgb_input(Y_cold,embeddings_cold)
        
        evalist = [(dmatrix_train,'train'),(dmatrix_cold,'eval')]
        xgb_model = class_XGB.build_model(dmatrix_train,evalist,300)
        xgb_pred_train = xgb_model.predict(dmatrix_train)
        xgb_pred_cold = xgb_model.predict(dmatrix_cold)
        
        if i < 6 :
            validation_metrics['Fold_%s'%i] = calculate_metrics(np.array(Y_cold),xgb_pred_cold)
            training_metrics['Fold_%s'%i] = calculate_metrics(np.array(Y),xgb_pred_train)
        elif i == 6:
            validation_metrics['Test'] = calculate_metrics(np.array(Y_cold),xgb_pred_cold)
            training_metrics['Test'] = calculate_metrics(np.array(Y),xgb_pred_train)
        


LAYER 0
LAYER 1
LAYER 2


C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2680 samples, validate on 537 samples
Epoch 1/25
2680/2680 [==============================] - 6s 2ms/step - loss: 0.9969 - val_loss: 0.9988
Epoch 2/25
2680/2680 [==============================] - 1s 281us/step - loss: 0.9961 - val_loss: 0.9970
Epoch 3/25
2680/2680 [==============================] - 1s 284us/step - loss: 0.9941 - val_loss: 0.9856
Epoch 4/25
2680/2680 [==============================] - 1s 284us/step - loss: 0.9888 - val_loss: 0.9818
Epoch 5/25
2680/2680 [==============================] - 1s 286us/step - loss: 0.9883 - val_loss: 0.9834
Epoch 6/25
2680/2680 [==============================] - 1s 280us/step - loss: 0.9810 - val_loss: 0.9841
Epoch 7/25
2680/2680 [==============================] - 1s 280us/step - loss: 0.9778 - val_loss: 0.9833
Epoch 8/25
2680/2680 [==============================] - 1s 286us/step - loss: 0.9773 - val_loss: 0.9820
Epoch 9/25
2680/2680 [==============================] - 1s 286us/step - loss: 0.9774 - val_loss: 0.9798
Epoch 10/25
2680/26

C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2680 samples, validate on 537 samples
Epoch 1/25
2680/2680 [==============================] - 6s 2ms/step - loss: 0.9967 - val_loss: 0.9986
Epoch 2/25
2680/2680 [==============================] - 1s 286us/step - loss: 0.9958 - val_loss: 0.9982
Epoch 3/25
2680/2680 [==============================] - 1s 284us/step - loss: 0.9950 - val_loss: 0.9970
Epoch 4/25
2680/2680 [==============================] - 1s 287us/step - loss: 0.9924 - val_loss: 0.9791
Epoch 5/25
2680/2680 [==============================] - 1s 279us/step - loss: 0.9852 - val_loss: 0.9751
Epoch 6/25
2680/2680 [==============================] - 1s 280us/step - loss: 0.9837 - val_loss: 0.9760
Epoch 7/25
2680/2680 [==============================] - 1s 296us/step - loss: 0.9785 - val_loss: 0.9644
Epoch 8/25
2680/2680 [==============================] - 1s 292us/step - loss: 0.9809 - val_loss: 0.9757
Epoch 9/25
2680/2680 [==============================] - 1s 286us/step - loss: 0.9791 - val_loss: 0.9775

Epoch 00009: Reduc

...

Train on 2680 samples, validate on 537 samples
Epoch 1/25
2680/2680 [==============================] - 9s 3ms/step - loss: 0.9967 - val_loss: 0.9979
Epoch 2/25
2680/2680 [==============================] - 1s 299us/step - loss: 0.9963 - val_loss: 0.9957
Epoch 3/25
2680/2680 [==============================] - 1s 286us/step - loss: 0.9946 - val_loss: 0.9827
Epoch 4/25
2680/2680 [==============================] - 1s 295us/step - loss: 0.9902 - val_loss: 0.9758
Epoch 5/25
2680/2680 [==============================] - 1s 301us/step - loss: 0.9859 - val_loss: 0.9748
Epoch 6/25
2680/2680 [==============================] - 1s 295us/step - loss: 0.9813 - val_loss: 0.9776
Epoch 7/25
2680/2680 [==============================] - 1s 298us/step - loss: 0.9840 - val_loss: 0.9749
Epoch 8/25
2680/2680 [==============================] - 1s 302us/step - loss: 0.9797 - val_loss: 0.9710
Epoch 9/25
2680/2680 [==============================] - 1s 306us/step - loss: 0.9770 - val_loss: 0.9700
Epoch 10/25
2680/26

C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2685 samples, validate on 532 samples
Epoch 1/25
2685/2685 [==============================] - 10s 4ms/step - loss: 0.9966 - val_loss: 0.9984
Epoch 2/25
2685/2685 [==============================] - 1s 292us/step - loss: 0.9961 - val_loss: 0.9971
Epoch 3/25
2685/2685 [==============================] - 1s 290us/step - loss: 0.9949 - val_loss: 0.9874
Epoch 4/25
2685/2685 [==============================] - 1s 288us/step - loss: 0.9923 - val_loss: 0.9828
Epoch 5/25
2685/2685 [==============================] - 1s 295us/step - loss: 0.9870 - val_loss: 0.9790
Epoch 6/25
2685/2685 [==============================] - 1s 291us/step - loss: 0.9851 - val_loss: 0.9700
Epoch 7/25
2685/2685 [==============================] - 1s 292us/step - loss: 0.9817 - val_loss: 0.9753
Epoch 8/25
2685/2685 [==============================] - 1s 291us/step - loss: 0.9840 - val_loss: 0.9768
Epoch 9/25
2685/2685 [==============================] - 1s 288us/step - loss: 0.9784 - val_loss: 0.9764
Epoch 10/25
2685/2

C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 3217 samples, validate on 537 samples
Epoch 1/25
3217/3217 [==============================] - 11s 3ms/step - loss: 0.9968 - val_loss: 0.9976
Epoch 2/25
3217/3217 [==============================] - 1s 297us/step - loss: 0.9962 - val_loss: 0.9909
Epoch 3/25
3217/3217 [==============================] - 1s 296us/step - loss: 0.9937 - val_loss: 0.9835
Epoch 4/25
3217/3217 [==============================] - 1s 289us/step - loss: 0.9872 - val_loss: 0.9676
Epoch 5/25
3217/3217 [==============================] - 1s 293us/step - loss: 0.9843 - val_loss: 0.9787
Epoch 6/25
3217/3217 [==============================] - 1s 292us/step - loss: 0.9823 - val_loss: 0.9738
Epoch 7/25
3217/3217 [==============================] - 1s 290us/step - loss: 0.9814 - val_loss: 0.9745
Epoch 8/25
3217/3217 [==============================] - 1s 291us/step - loss: 0.9789 - val_loss: 0.9747
Epoch 9/25
3217/3217 [==============================] - 1s 293us/step - loss: 0.9753 - val_loss: 0.9816
Epoch 10/25
3217/3

In [11]:
pd.DataFrame(validation_metrics).T

,roc_auc,tn,fp,fn,tp,map,precision,recall,accuracy
Fold_0,0.808145,275.0,54.0,72.0,136.0,0.710781,0.715789,0.653846,0.765363
Fold_1,0.849271,291.0,76.0,41.0,129.0,0.716986,0.629268,0.758824,0.782123
Fold_2,0.852865,293.0,46.0,57.0,141.0,0.773155,0.754011,0.712121,0.808194
Fold_3,0.843115,317.0,57.0,65.0,98.0,0.733481,0.632258,0.601227,0.772812
Fold_4,0.899018,299.0,43.0,60.0,135.0,0.810340,0.758427,0.692308,0.808194
Fold_5,0.842660,185.0,91.0,36.0,220.0,0.802736,0.707395,0.859375,0.761278
Test,0.841000,303.0,57.0,55.0,122.0,0.678572,0.681564,0.689266,0.791434


In [12]:
pd.DataFrame(training_metrics).T

,roc_auc,tn,fp,fn,tp,map,precision,recall,accuracy
Fold_0,0.952009,1499.0,199.0,100.0,882.0,0.899147,0.815911,0.898167,0.888433
Fold_1,0.941182,1442.0,218.0,120.0,900.0,0.897686,0.805009,0.882353,0.873881
Fold_2,0.941510,1494.0,194.0,132.0,860.0,0.896704,0.815939,0.866935,0.878358
Fold_3,0.919076,1440.0,213.0,182.0,845.0,0.868734,0.798677,0.822785,0.852612
Fold_4,0.941236,1491.0,194.0,146.0,849.0,0.893681,0.813998,0.853266,0.873134
Fold_5,0.939323,1554.0,197.0,145.0,789.0,0.877803,0.800203,0.844754,0.872626
Test,0.938083,1786.0,241.0,174.0,1016.0,0.886696,0.808274,0.853782,0.870998
